# verdicthub

### Data Cleaning

In [ ]:
import pandas as pd
data = pd.read_csv(r"C:\Users\cyril\Pictures\GAIP\Project\Dataset\Judgment_minimal.csv")  # Assuming data is in a CSV file
data.info()

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data["BACKGROUND TO THE APPEAL"], data["JUDGMENT_MINIMAL"], test_size=0.2, random_state=42)


In [3]:
X_test.tolist()[0]

'lord neuberger president lord mance lord clarke lord sumption lord hodge background appeal appeal arises fact foundation structure two offshore wind farm robin rigg solway firth designed installed respondent mt hjgaard mth failed shortly companypletion project dispute companycerns bear remedial companyts sum may appellant two companypanies eon group eon sent tender document various party including mth due companyrse became successful bidder tender document included eon technical requirement technical requirement laid minimum requirement taken account companytractor ie ultimately mth amongst thin g technical requirement called foundation accordance document known j j reference international standard design offshore wind turbine published independent classification certification agency j provides certain mathematical formula calculate aspect foundation structure one formula included given specific value later review showed value given wrong factor ten error meant strength foundation str

In [4]:
import torch
import pandas as pd
from torch.utils.data import DataLoader
df = pd.DataFrame()

# Load and preprocess data
#data = pd.read_csv(r"/content/Judgment_minimal.csv")  # Assuming data is in a CSV file
#data = data.dropna()

#df["BACKGROUND TO THE APPEAL"] = data["BACKGROUND TO THE APPEAL"].apply(clean_text)
#df["JUDGMENT"] = data["JUDGMENT"].apply(clean_text)

#data["BACKGROUND TO THE APPEAL"] = data["BACKGROUND TO THE APPEAL"].apply(str.lower)  # Example preprocessing
#data["ORIGINAL JUDGMENT"] = data["ORIGINAL JUDGMENT"].apply(str.lower)

# Prepare input and target examples
input_texts = X_train #data["BACKGROUND TO THE APPEAL"].tolist()
target_texts = y_train #data["JUDGMENT_MINIMAL"].tolist()

# Create DataLoader with appropriate batch size
batch_size = 1  # Adjust as needed
data_loader = DataLoader(list(zip(input_texts, target_texts)), batch_size=batch_size, shuffle=True)



In [5]:
!pip install SentencePiece

### t5-small

In [6]:
from transformers import AutoModelForSeq2SeqLM, T5ForConditionalGeneration, T5Tokenizer, AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [21]:
model_name = "t5-small"  # Choose a suitable transformer model
#model_name = "t5-large"
#model_name = "eleutheral/falcon-7b"
tokenizer = T5Tokenizer.from_pretrained(model_name, model_max_length=1024)
model = T5ForConditionalGeneration.from_pretrained(model_name)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Pegasus-xsum

In [5]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = "google/pegasus-xsum"

tokenizer = PegasusTokenizer.from_pretrained(model_name, max_length = 1024)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Models Tryed to run in system

In [ ]:
from transformers import GPTJForCausalLM, AutoTokenizer

model = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

# Load model and tokenizer
model = PegasusForConditionalGeneration.from_pretrained("nsi319/legal-pegasus")  # Adjust model name as needed
tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-pegasus")

In [ ]:
from transformers import T5ForConditionalGeneration, AutoTokenizer, DataCollatorForSeq2Seq
from torch.utils.data import DataLoader

# Load model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("google/t5-base")  # Adjust model name as needed
tokenizer = AutoTokenizer.from_pretrained("google/t5-base")

### Gpu


In [22]:
# Move model to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
model = model.to(device)

In [23]:
# Set up optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)  # Adjust learning rate if needed
loss_fn = torch.nn.CrossEntropyLoss()

In [24]:
for epoch in range(10):
    epoch_loss = 0.0

    for batch_idx, (input_text, target_text) in enumerate(data_loader):
        # Tokenize input and target text
        encoding = tokenizer(input_text, padding=True, truncation=True, max_length=512, return_tensors="pt")
        input_ids = encoding.input_ids
        attention_mask = encoding.attention_mask

        target_ids = tokenizer(target_text, padding=True, truncation=True, max_length=512, return_tensors="pt").input_ids

        # Move the tensors to the device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        target_ids = target_ids.to(device)

        # Get model outputs
        outputs = model(input_ids, attention_mask=attention_mask, labels=target_ids)
        #print(model)

        # Compute the loss
        loss = loss_fn(outputs.logits.view(-1, model.config.vocab_size), target_ids.view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{20}, Loss: {epoch_loss / len(input_texts):.4f}")



Epoch 1/20, Loss: 4.0782
Epoch 2/20, Loss: 3.5708
Epoch 3/20, Loss: 3.3553
Epoch 4/20, Loss: 3.1665
Epoch 5/20, Loss: 2.9820
Epoch 6/20, Loss: 2.7898
Epoch 7/20, Loss: 2.5853
Epoch 8/20, Loss: 2.3964
Epoch 9/20, Loss: 2.1747
Epoch 10/20, Loss: 1.9799


In [11]:
input_prompt = "Case appeal: [lord rodger lord brown lord mance lord collins lord dyson background appeal issue appeal whether court ppeal right order retrial respect appellant circumstance companyrt may order retrial set section criminal appeal act amended crim inal justice act provides court appeal allows appeal companyviction appears court interest justice require may order appellant retried appellant brother companyvicted mu rder two robbery leeds crown court february appellant sentenced life imprisonment murder served companycurrent twelveyear term robbery main prosecution witness karl chapman professional criminal supergrass june october robbery took place home two elderly brother occasion th e robber used violence took money second occasion elder brother sustained injury head later resulted death ensuing police investigation chapman provided police information witness statement implicating appellant brother charged robbery murder chapman evidence central prosecution case tria l vigorously denied expecting receiving benefit police evidence following companyvictions allegation local press police planning pay chapman large sum money upon release prison subsequently crim inal case review commission ccrc decided investigate north yorkshire police carried detaile investigation activity police formed basis ccrc report nove mber finding report showed police companyspired pervert companyrse justice companycealing lying variety reward benefit received chapman vealed example police paid sum money taken brothel allowed companysume drug companypany number investigated allegation companymitted viol ent attack november ccrc made reference criminal divisi court appeal ground companyvictions procured gross prosecutorial misconduct part police appellant brother remained prison period october september whilst prison appellant made series admission guilt freely voluntarily various person december court appeal quashed companyvictions appellant brother finding ccrc relating gross po louse misconduct number challenged companyrt held finding revealed trial trial judge might stayed prosecution abuse process applied section th e police criminal evidence act exclude chapman evidence altogether case th e appellant brother would acquitted however companyrt also held ad mission made appellant companystituted clear companypelling evidence gu ilt light court appeal found interest justice order retrial appellant supreme court united kingdom parliament square london swp bd f wwwsupremecourtgovuk]\nVerdict:"
input_ids = tokenizer.encode(input_prompt,truncation=True, max_length=512, return_tensors="pt")
input_ids = input_ids.to(model.device)

generated_verdict = model.generate(input_ids, max_length=512, num_return_sequences=1)[0]
print(generated_verdict)

tensor([    0,    37,  8531,  2243, 16708,    15,     7,     8,  3958,    57,
            3,     9,  2942,    13,     3,    18,     6,    28,     8,   380,
           13,  2809, 12991,   739,    16,     8,   962,    13,   823,     8,
         3958,    19,  2016,   242,     8,  3958,     8,  1046,  1614,   398,
          455,     3,     9,     3,    60,    17, 12042,     3,    99,    34,
           19,  2016,    21,     8,  8319,  6761,     7,  4616,  2809,   283,
          663,  1527,     8,   991,  7661,     6,    28,    84,  2809, 12991,
          739,  2065,     7,  2809,   283,   663,  1527,     3,     9,     3,
           26, 10924,    53,  7661,     3, 13223, 10466,   134,  5652,  1853,
          446, 10161,   517, 11810,    37,     3, 27794,  6761,     7, 24062,
           19,    59,  2303,    57,     8,     3,    75,    40,     3,    52,
           26,     7, 24062,    11,     3,    23,    23,     8,  4616,     3,
            6,    68,     3,    23,   208,     8,  4616,    16, 

In [12]:
# Convert the tensor back to CPU if necessary
decoded_text = tokenizer.decode(generated_verdict.cpu(), skip_special_tokens=True)

# Print the generated verdict
print(decoded_text)


The Supreme Court dismisses the appeal by a majority of -, with the support of Lord Dyson in the issue of whether the appeal is appropriate For the appeal the interest court must order a retrial if it is appropriate for the appellants circumstances Lord Mance gives the lead judgment, with which Lord Dyson agrees Lord Mance gives a dissenting judgment REASONS FOR THE JUDGMENT The Appellants circumstance is not covered by the cl rds circumstance and ii the circumstances, but iv the circumstances in which the appellants were convicted, and the court is not obliged to exclude chapman evidence from the court s decision,, Lord Brown and Lord Dyson in the issue of whether the court has a trump of contention that the interest court is obliged to order a retrial, but that the interest court has a right to order a retrial, and that the interest court has a right to order a retrial, and that the interest court has a right to order a retrial, Lord Mance agrees with Lord Dyson in his view of the is

In [25]:
# Save the model and tokenizer
model.save_pretrained('/content/drive/MyDrive/t5-small-model')
tokenizer.save_pretrained('/content/drive/MyDrive/t5-small-model')

('/content/drive/MyDrive/t5-small-model/tokenizer_config.json',
 '/content/drive/MyDrive/t5-small-model/special_tokens_map.json',
 '/content/drive/MyDrive/t5-small-model/spiece.model',
 '/content/drive/MyDrive/t5-small-model/added_tokens.json')

### Save Model

In [ ]:
!pip install sentencepiece

In [19]:
# Specify the directory where you want to save the model
output_dir = "/content/drive/MyDrive/T5-small_model"

# Save the model
model.save_pretrained(output_dir)

# Save the tokenizer
tokenizer.save_pretrained(output_dir)


('/content/drive/MyDrive/T5-small_model/tokenizer_config.json',
 '/content/drive/MyDrive/T5-small_model/special_tokens_map.json',
 '/content/drive/MyDrive/T5-small_model/spiece.model',
 '/content/drive/MyDrive/T5-small_model/added_tokens.json',
 '/content/drive/MyDrive/T5-small_model/tokenizer.json')

In [15]:
!pip install -r "/content/requirements (1).txt"

In [16]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=daacfaa2eca8375eedfdc89e0348f436cc6dc562abb88c25197d0df73375c0b4
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


### Testing and evaluation

#### Rouge

In [32]:
from rouge_score import rouge_scorer
scorer=rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL','rougeLsum'])
scores = scorer.score(' The Supreme Court dismisses the appeal by a majori ty of Lord Dyson gives the lead judgment Lords Rodger and Mance give short concurring judgments Lords Brown and Collins dissent On November the Supreme Court handed down its decision but withheld its reasoning until the completion of the retrial On June , in Leeds Crown Court Paul Maxwell pleaded guilty REASONS FOR THE JUDGMENT By section of the Criminal Appeal Act , as am ended, Parliament has given the Criminal Division of the Court of Appeal the power to order a retrial where, having rega rd to all the circumstances of the particular case, in its view the interests of justice so require The interests of justice is not a hard-edged concept Rather, it requires an exercise of ju dgment in which a number of relevant factors have to be balanced against each other A decision of th e Court of Appeal as to whether the interests of justice require a retrial should only be upset on appeal if it was plainly wrong in the sense that it is one which no reasonable court could have made or if the court took into account immaterial factors or failed to take into account material factors - The majority of cases under section of the Act do not involve any issue of prosecutorial misconduct Indeed, no case was cited where the court had to consider the relevance of prosecutori al misconduct in the original proceedings to the question of whether the interests of justice requir e a retrial Usually, under section the court will consider the gravity of the offence, the length of time the appellant is likely to serve in custody if reconvicted, the appellants age and health, and th e wishes of the victim of the alleged offence Where prosecutorial misconduct is involved the Court of Appeal may treat the case as to some extent analogous to an application to stay proceedings as an abuse of process where it offends the courts sense of justice and propriety to try the accused However, the tests for when the court should stay proceedings for abuse of process and when it shou ld order a retrial are not coterminous The question of whether the interests of justice require a retrial is broader than the considerations involved in an application for a stay , It is common ground that the prosecutions case at a retrial would not be based on any evidence which was the product of the misconduct However, the new evidence constitutes admissions made by the appellant which would not have been made but for the original misconduct which led to his conviction The Court of Appeal was right to consider that the but for factor was no more than a relevant factor and that it was not determinative of the question whether a retrial was required in the interests of justice In deciding whether to order a retrial, there were several relevant factors which had to be weighed in the balance The balancing act is fact-sensitive and ultimately requires an exercise of judgment The Court of Appeal carri ed out the balancing exercise preci sely and with great care They held that there were strong reasons for not orderi ng a retrial given the egregious misconduct by the police However, they concluded that the public interes t in convicting those guilty of murder prevailed on the facts of this case In particular this was be cause of the gravity of the alleged offence and the existence of new and compelling evidence untainted by the police misconduct The fact that a differently constituted Court of Appeal might have come to a different conclusion is not material Accordingly, the decision of the Court of Appeal was not plainly wrong and its judgment should not be interfered with -, -, - Lord Brown, with whom Lord Collins agrees, would have allowed the appeal They would have held that since the appellant would not have made the admissions but for the prosecutorial misconduct and in light of the enormity of the police misconduct, it is inappropriate that that the case should be retried on new evidence - References in square brackets are to paragraph numbers in the judgment  ',
                      'The Supreme Court dismisses the appeal by a majori ty of Lord Dyson gives the lead judgment Lords Rodger and Mance give short concurring judgments Lords Brown and Collins dissent On November the Supreme Court handed down its decision but withheld its reasoning until the completion of the retrial On June, in Leeds Crown Court Paul Maxwell pleaded guilty REASONS FOR THE JUDGMENT By section of the Criminal Appeal Act, as am ended, Parliament has given the Criminal Division of the Court of Appeal the power to order a retrial where, having rega rd to all the circumstances of the particular case, in its view the interests of justice so require The interests of justice is not a hard-edged concept Rather, it requires an exercise of ju dgment in which a number of relevant factors have to be balanced against each other A decision of th e Court of Appeal as to whether the interests of justice require a retrial should only be upset on appeal if it was plainly wrong in the sense that it is one which no reasonable court could have made or if the court took into account immaterial factors or failed to take into account material factors - The majority of cases under section of the Act do not involve any issue of prosecutorial misconduct Indeed, no case was cited where the court had to stay proceedings simply because the prosecution or the defence argued that the judge should stay out of it Alleg ations of prosecutorial misconduct are, however, relevant to the question of whether the interests of justice require a stay of proceedings where, having rega rd to all the circumstances of the particular case, it would not be right for the court to stay out of a retrial where, having rega rd to all the circumstances of the particular case, the court would not be right to stay out of it No case was cited where the court might stay proceedings in the interests of justice where the only potentially relevant factor was the gravity of the offence committed by the defendant However, as a matter of principle the court should not stay proceedings it out of the interests of justice where, having rega rd to all the circumstances of the particular case, it would not be right to stay out of a trial in crim inal proceedings where the only potentially relevant factor was the gravity of the offence committed by the defendant However, as a matter of principle the court should not stay proceedings in crim inal proceedings where, having rega rd to all the circumstances of the particular case, it would not be right to stay proceedings in crim inal proceedings')
print(scores)

{'rouge1': Score(precision=0.8230088495575221, recall=0.533715925394548, fmeasure=0.6475195822454308), 'rouge2': Score(precision=0.6319290465631929, recall=0.40948275862068967, fmeasure=0.4969485614646906), 'rougeL': Score(precision=0.6902654867256637, recall=0.44763271162123386, fmeasure=0.5430809399477807), 'rougeLsum': Score(precision=0.6902654867256637, recall=0.44763271162123386, fmeasure=0.5430809399477807)}


In [41]:
from rouge_score import rouge_scorer
scores = []
for i in range(len(X_test)):
    u = "Case appeal: ["+X_test.tolist()[i]+"]"
    input_ids = tokenizer.encode(u,truncation=True, max_length=512, return_tensors="pt")
    input_ids = input_ids.to(model.device)
    generated_verdict = model.generate(input_ids, max_length=512, num_return_sequences=1)[0]

    decoded_text = tokenizer.decode(generated_verdict.cpu(), skip_special_tokens=True)

    scorer=rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL','rougeLsum'])
    scores1 = scorer.score(y_test.tolist()[i],decoded_text)
    scores.append(scores1)
    print(scores1)

{'rouge1': Score(precision=0.5067024128686327, recall=0.4144736842105263, fmeasure=0.4559710494571773), 'rouge2': Score(precision=0.11290322580645161, recall=0.09230769230769231, fmeasure=0.10157194679564692), 'rougeL': Score(precision=0.2225201072386059, recall=0.18201754385964913, fmeasure=0.20024125452352234), 'rougeLsum': Score(precision=0.2225201072386059, recall=0.18201754385964913, fmeasure=0.20024125452352234)}
{'rouge1': Score(precision=0.6153846153846154, recall=0.15568862275449102, fmeasure=0.24850657108721624), 'rouge2': Score(precision=0.20833333333333334, recall=0.05247376311844078, fmeasure=0.08383233532934131), 'rougeL': Score(precision=0.3727810650887574, recall=0.09431137724550898, fmeasure=0.15053763440860216), 'rougeLsum': Score(precision=0.3727810650887574, recall=0.09431137724550898, fmeasure=0.15053763440860216)}
{'rouge1': Score(precision=0.6419753086419753, recall=0.07407407407407407, fmeasure=0.13282247765006386), 'rouge2': Score(precision=0.1875, recall=0.021

In [45]:
total_precision = 0
total_recall=0
total_fmeasure=0
count = 0

for score_dict in scores:
    if "rouge1" in score_dict:
        total_precision += score_dict["rouge1"].precision
        total_recall += score_dict["rouge1"].recall
        total_fmeasure += score_dict["rouge1"].fmeasure
        count += 1

average_precision = total_precision / count if count > 0 else 0
average_recall = total_recall / count if count > 0 else 0
average_fmeasure = total_fmeasure / count if count > 0 else 0

print("Average Rouge1 Precision:", average_precision)
print("Average Rouge1 Recall:", average_recall)
print("Average Rouge1 F-Measure:", average_fmeasure)

Average Rouge1 Precision: 0.5188299115380309
Average Rouge1 Recall: 0.27405336924176965
Average Rouge1 F-Measure: 0.3480278489893209


In [46]:
total_precision = 0
total_recall=0
total_fmeasure=0
count = 0

for score_dict in scores:
    if "rouge2" in score_dict:
        total_precision += score_dict["rouge2"].precision
        total_recall += score_dict["rouge2"].recall
        total_fmeasure += score_dict["rouge2"].fmeasure
        count += 1

average_precision = total_precision / count if count > 0 else 0
average_recall = total_recall / count if count > 0 else 0
average_fmeasure = total_fmeasure / count if count > 0 else 0

print("Average Rouge2 Precision:", average_precision)
print("Average Rouge2 Recall:", average_recall)
print("Average Rouge2 F-Measure:", average_fmeasure)

Average Rouge2 Precision: 0.14812283118982691
Average Rouge2 Recall: 0.07589086507374372
Average Rouge2 F-Measure: 0.09711551888467784


In [47]:
total_precision = 0
total_recall=0
total_fmeasure=0
count = 0

for score_dict in scores:
    if "rougeL" in score_dict:
        total_precision += score_dict["rougeL"].precision
        total_recall += score_dict["rougeL"].recall
        total_fmeasure += score_dict["rougeL"].fmeasure
        count += 1

average_precision = total_precision / count if count > 0 else 0
average_recall = total_recall / count if count > 0 else 0
average_fmeasure = total_fmeasure / count if count > 0 else 0

print("Average RougeL Precision:", average_precision)
print("Average RougeL Recall:", average_recall)
print("Average RougeL F-Measure:", average_fmeasure)

Average RougeL Precision: 0.2736098138632765
Average RougeL Recall: 0.14040438306097427
Average RougeL F-Measure: 0.17940789518890388


In [28]:
total_precision = 0
total_recall=0
total_fmeasure=0
count = 0

for score_dict in scores:
    if "rougeLsum" in score_dict:
        total_precision += score_dict["rougeLsum"].precision
        total_recall += score_dict["rougeLsum"].recall
        total_fmeasure += score_dict["rougeLsum"].fmeasure
        count += 1

average_precision = total_precision / count if count > 0 else 0
average_recall = total_recall / count if count > 0 else 0
average_fmeasure = total_fmeasure / count if count > 0 else 0

print("Average RougeLsum Precision:", average_precision)
print("Average RougeLsum Recall:", average_recall)
print("Average RougeLsum F-Measure:", average_fmeasure)

Average RougeLsum Precision: 0.26862981484739407
Average RougeLsum Recall: 0.15002831546328851
Average RougeLsum F-Measure: 0.18690166593355823


#### Meteor

In [54]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [88]:
from nltk.translate import meteor
from nltk.tokenize import word_tokenize
scores = []
for i in range(len(X_test)):
    u = "Case appeal: ["+X_test.tolist()[i]+"]"
    input_ids = tokenizer.encode(u,truncation=True, max_length=512, return_tensors="pt")
    input_ids = input_ids.to(model.device)
    generated_verdict = model.generate(input_ids, max_length=512, num_return_sequences=1)[0]

    decoded_text = tokenizer.decode(generated_verdict.cpu(), skip_special_tokens=True)
    reference = word_tokenize(y_test.iloc[i])
    decoded_text = word_tokenize(decoded_text)
    meteor_score = round(meteor([decoded_text],reference), 4)
    scores.append(meteor_score)
    print(scores)

[0.1971]
[0.1971, 0.2183]
[0.1971, 0.2183, 0.3153]
[0.1971, 0.2183, 0.3153, 0.2586]
[0.1971, 0.2183, 0.3153, 0.2586, 0.2575]
[0.1971, 0.2183, 0.3153, 0.2586, 0.2575, 0.2342]
[0.1971, 0.2183, 0.3153, 0.2586, 0.2575, 0.2342, 0.2757]
[0.1971, 0.2183, 0.3153, 0.2586, 0.2575, 0.2342, 0.2757, 0.3173]
[0.1971, 0.2183, 0.3153, 0.2586, 0.2575, 0.2342, 0.2757, 0.3173, 0.2567]
[0.1971, 0.2183, 0.3153, 0.2586, 0.2575, 0.2342, 0.2757, 0.3173, 0.2567, 0.2729]
[0.1971, 0.2183, 0.3153, 0.2586, 0.2575, 0.2342, 0.2757, 0.3173, 0.2567, 0.2729, 0.2443]
[0.1971, 0.2183, 0.3153, 0.2586, 0.2575, 0.2342, 0.2757, 0.3173, 0.2567, 0.2729, 0.2443, 0.3166]
[0.1971, 0.2183, 0.3153, 0.2586, 0.2575, 0.2342, 0.2757, 0.3173, 0.2567, 0.2729, 0.2443, 0.3166, 0.2122]
[0.1971, 0.2183, 0.3153, 0.2586, 0.2575, 0.2342, 0.2757, 0.3173, 0.2567, 0.2729, 0.2443, 0.3166, 0.2122, 0.2748]
[0.1971, 0.2183, 0.3153, 0.2586, 0.2575, 0.2342, 0.2757, 0.3173, 0.2567, 0.2729, 0.2443, 0.3166, 0.2122, 0.2748, 0.3107]
[0.1971, 0.2183, 0.3153, 

In [1]:
sum(scores)/len(scores)

NameError: ignored

### Load the model

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.3 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForSeq2SeqLM, T5Tokenizer, AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [14]:
# Specify the directory where you want to save the model
model_dir = "/content/drive/MyDrive/t5-small-model"
tokenizer = T5Tokenizer.from_pretrained(model_dir, model_max_length=2048)
model = T5ForConditionalGeneration.from_pretrained(model_dir)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Specify the directory where you want to save the model
model_dir = "/content/drive/MyDrive/Pegasus_model"

# Load the model
model = PegasusForConditionalGeneration.from_pretrained(model_dir)

# Load the tokenizer
tokenizer = PegasusTokenizer.from_pretrained(model_dir,model_max_length=512)


In [5]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [15]:
input_prompt = "Case appeal: [The appeal concerns the dismissal of Ms Jhuti, the appellant, from her employment by Royal Mail Group Ltd (“the company”). The key question of law that it raises is as follows: in a claim for unfair dismissal under Part X of the Employment Rights Act 1996 (“the Act”), can the reason for the dismissal be other than that given to the employee by the employer’s appointed decision-maker? The facts found by the employment tribunal (“the tribunal”) in this case included the following. During her trial period, Ms Jhuti made ‘protected disclosures’ under section 43A of the Act, commonly described as whistleblowing. Her line manager’s response was to pretend that her performance was inadequate, including by bullying her and by creating, in emails and otherwise, a false picture of her performance. The company appointed another employee to decide whether Ms Jhuti should be dismissed. Ms Jhuti, who had in the meantime been signed off work for work-related stress, anxiety and depression, was unable to present her case to the decision-maker in meetings or otherwise. Having no reason to doubt the truthfulness of the material indicative of Ms Jhuti’s inadequate performance, the decision-maker decided that she should be dismissed for that reason. Ms Jhuti brought two complaints in the tribunal. The first complaint (on which nothing in the present appeal turns directly) was that, contrary to section 47B(1) of the Act, she had been subjected to detriments by acts of the company done on the ground of her whistleblowing. The second complaint was that her dismissal was unfair under section 103A, which provides that a dismissal is unfair ‘if the reason (or, if more than one, the principal reason) for the dismissal is that the employee made a protected disclosure’. The tribunal dismissed this second complaint. It found that, as the decision-maker had dismissed her on the ground of a genuine belief that her performance had been inadequate, the reason for dismissal was her performance and so section 103A did not apply. The Employment Appeal Tribunal (“the EAT”) reversed this decision, holding that the reason for dismissal was the making of the protected disclosures. The Court of Appeal allowed the company’s appeal against the EAT’s decision and reinstated the tribunal’s dismissal of the complaint of unfair dismissal. It held that a tribunal required to determine the reason for dismissal under section 103A was obliged to consider only the mental processes of the employer’s authorised decision-maker. Ms Jhuti appealed to the Supreme Court.]\nVerdict:"
input_ids = tokenizer.encode(input_prompt,truncation=True, max_length=1024, return_tensors="pt")
input_ids = input_ids.to(model.device)

generated_verdict = model.generate(input_ids, max_length=1024, num_return_sequences=1)[0]
print(generated_verdict)

tensor([    0,    37, 20512, 24910, 19664,  8667,   446, 13985,    23,    45,
          160,  4311,    44,     8,    97,    13,   160,  3689,     3,     5,
           37, 23698, 19664,     8, 10394,    13, 18216, 16708,   138,    44,
            8,    97,    13,   160,  3689,     3,     5,   283,     7,   446,
        13985,    23,    47,  1426,    15,    26,    12,    20,  1788,  4128,
           57,  6775,    13,     8,   349,    30,     8,  9808,    13,   160,
        23585,  4672,  3108,     3,     5,  5568,     3, 17864,   188,   405,
           59,  1581,    12,     3,     9, 16708,   138,     3,  6475,   283,
            7,   446, 13985,    23,     3,     5,     1])


In [16]:
# Convert the tensor back to CPU if necessary
decoded_text = tokenizer.decode(generated_verdict.cpu(), skip_special_tokens=True)

# Print the generated verdict
print(decoded_text)

The Employment Tribunal dismissed Mrs Jhuti from her employment at the time of her trial. The tribunal dismissed the complaint of unfair dismissal at the time of her trial. Ms Jhuti was subjected to detriments by acts of the company on the grounds of her whistleblowing. Section 103A does not apply to a dismissal involving Ms Jhuti.
